Code to get pix2pixzero

In [ ]:
!git clone https://github.com/pix2pixzero/pix2pix-zero.git

In [ ]:
!pip install torchvision

In [ ]:
!pip install accelerate diffusers==0.12.0 einops gradio ipython numpy opencv-python-headless pillow psutil tqdm transformers salesforce-lavis

Define Helper Functions to invert the image and create the semantic edit direction (setup phase), as well as create baseline images (sans mask) and images with the new loss function (using mask)

In [ ]:
import os
def create_inversion(num):
  s = subprocess.getstatusoutput(f"cd pix2pix-zero; python3 src/inversion.py --input_image \"../{num} copy.png\" --results_folder \"output/all_data_inv\" --use_float_16")
  # if s[0] == 0:
  #   print(s[1])
  # else:
  #     print('Custom Error {}'.format(s[1]))

In [ ]:
import subprocess
def generate_y2o_photo(num):
  s = subprocess.getstatusoutput(f'cd pix2pix-zero ; python3 src/edit_real.py --inversion "output/all_data_inv/inversion/{num} copy.pt" --prompt "output/all_data_inv/prompt/{num} copy.txt" --task_name "young2old" --results_folder "output/AA_rep_y2o"  --use_float_16')
  # if s[0] == 0:
  #   print(s[1])
  # else:
  #     print('Custom Error {}'.format(s[1]))
def generate_b2b_photo(num):
  s = subprocess.getstatusoutput(f'cd pix2pix-zero ; python3 src/edit_real.py --inversion "output/all_data_inv/inversion/{num} copy.pt" --prompt "output/all_data_inv/prompt/{num} copy.txt" --task_name "brunette2blonde" --results_folder "output/AA_rep_b2b"  --use_float_16')
  # if s[0] == 0:
  #   print(s[1])
  # else:
  #     print('Custom Error {}'.format(s[1]))
def y2o_base(num):
  s = subprocess.getstatusoutput(f'cd pix2pix-zero ; python3 src/edit_real.py --inversion "output/all_data_inv/inversion/{num} copy.pt" --prompt "output/all_data_inv/prompt/{num} copy.txt" --task_name "young2old" --results_folder "output/AA_rep_y2o_base"  --use_float_16')
  # if s[0] == 0:
  #   print(s[1])
  # else:
  #     print('Custom Error {}'.format(s[1]))
def b2b_base(num):
  s = subprocess.getstatusoutput(f'cd pix2pix-zero ; python3 src/edit_real.py --inversion "output/all_data_inv/inversion/{num} copy.pt" --prompt "output/all_data_inv/prompt/{num} copy.txt" --task_name "brunette2blonde" --results_folder "output/AA_rep_b2b_base"  --use_float_16')
  # if s[0] == 0:
  #   print(s[1])
  # else:
  #     print('Custom Error {}'.format(s[1]))

Helper functions to load the masks

In [ ]:
import cv2
import numpy as np
import pickle
# Load the image

def create_y2o_mask(num):
  image1 = cv2.imread(str(num).zfill(5) + "_hair copy.png")

  image2 = cv2.imread(str(num).zfill(5) + "_skin copy.png")

  # Convert the image to a NumPy array
  image_array = np.array(image1)
  image_array2 = np.array(image2)

  final_mask = image_array / 255 + image_array2 / 255

  for i in range(512):
    for j in range(512):
      for k in range(3):
        if final_mask[i,j,k] >= 1:
          final_mask[i,j,k] = 1
        final_mask[i,j,k] = int(final_mask[i,j,k])




  # pickle the object
  with open("./pix2pix-zero/current_mask.pkl", 'wb+') as f:
      pickle.dump(final_mask, f)

def create_b2b_mask(num):
  image1 = cv2.imread(str(num).zfill(5) + "_hair copy.png")

  # Convert the image to a NumPy array
  image_array = np.array(image1)

  final_mask = image_array / 255

  for i in range(512):
    for j in range(512):
      for k in range(3):
        if final_mask[i,j,k] >= 1:
          final_mask[i,j,k] = 1
        final_mask[i,j,k] = int(final_mask[i,j,k])

  # pickle the object
  with open("./pix2pix-zero/current_mask.pkl", 'wb+') as f:
      pickle.dump(final_mask, f)


Loop to create the images

In [ ]:
for i in range(12, 100):
  print(i)
  create_inversion(i)
  with open("./pix2pix-zero/lamb.pkl", 'wb+') as f:
    lamb = .01
    pickle.dump(lamb, f)
  create_y2o_mask(i)
  generate_y2o_photo(i)
  create_b2b_mask(i)
  generate_b2b_photo(i)
  with open("./pix2pix-zero/lamb.pkl", 'wb+') as f:
    lamb = 0
    pickle.dump(lamb, f)
  y2o_base(i)
  b2b_base(i)

Zip the created images for download

In [ ]:
!zip -r './pix2pix-zero/output/AA_rep_b2b' './pix2pix-zero/output/AA_rep_b2b'

In [ ]:
!zip -r './pix2pix-zero/output/AA_rep_y2o' './pix2pix-zero/output/AA_rep_y2o'

In [ ]:
!zip -r './pix2pix-zero/output/AA_rep_b2b_base' './pix2pix-zero/output/AA_rep_b2b_base'

In [ ]:
!zip -r './pix2pix-zero/output/AA_rep_y2o_base' './pix2pix-zero/output/AA_rep_y2o_base'

In [ ]:
!cd pix2pix-zero; python src/make_edit_direction.py --file_source_sentences sentences/brunette.txt --file_target_sentences sentences/blonde.txt --output_folder assets/embeddings_sd_1.4

The below generates the requisite .pt files for the old2young and brunette2blond tasks using the chatGPT generated sentences  

In [ ]:
!cd pix2pix-zero; python src/make_edit_direction.py --file_source_sentences sentences/old.txt --file_target_sentences sentences/young.txt --output_folder assets/embeddings_sd_1.4

In [ ]:
!cd pix2pix-zero; python src/make_edit_direction.py --file_source_sentences sentences/blonde.txt --file_target_sentences sentences/brunette.txt --output_folder assets/embeddings_sd_1.4